In [1]:
pip install gradio


In [2]:
# !git clone --single-branch --branch gptq-4bit-64g-actorder_True https://huggingface.co/TheBloke/Llama-2-7b-Chat-GPTQ
# !git clone --single-branch --branch gptq-4bit-32g-actorder_True https://huggingface.co/TheBloke/zephyr-7B-beta-GPTQ


In [3]:
# !pip install transformers torch
# !pip install --upgrade transformers accelerate

In [4]:
# !pip install transformers optimum
# !pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

In [5]:
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# model_name_or_path = "TheBloke/Llama-2-7b-Chat-GPTQ"
# # To use a different branch, change revision
# # For example: revision="gptq-4bit-64g-actorder_True"
# model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
#                                              device_map="auto",
#                                              trust_remote_code=False,
#                                              revision="main")

# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

# prompt = "I am data scientist having expertise in nlp and computer vision both."
# prompt_template=f'''[INST] <<SYS>>
# You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
# <</SYS>>
# {prompt}[/INST]

# '''

# print("\n\n*** Generate:")

# input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
# output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
# print(tokenizer.decode(output[0]))

# # Inference can also be done using transformers' pipeline

# print("*** Pipeline:")
# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=512,
#     do_sample=True,
#     temperature=0.7,
#     top_p=0.95,
#     top_k=40,
#     repetition_penalty=1.1
# )

# print(pipe(prompt_template)[0]['generated_text'])


In [6]:
# prompt = "I am data scientist having expertise in nlp and computer vision both."
# prompt_template=f'''[INST] <<SYS>>
# You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
# <</SYS>>
# {prompt}[/INST]

# '''

# print("\n\n*** Generate:")

# input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
# output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
# print(tokenizer.decode(output[0]))

# # Inference can also be done using transformers' pipeline

# print("*** Pipeline:")
# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=512,
#     do_sample=True,
#     temperature=0.7,
#     top_p=0.95,
#     top_k=40,
#     repetition_penalty=1.1
# )
# response = pipe(prompt_template)[0]['generated_text']
# # print(pipe(prompt_template)[0]['generated_text'])

In [7]:
# full_text = response

# # Find the indices of "[INST]" and "[/INST]"
# start_index = full_text.find("[INST]")
# end_index = full_text.find("[/INST]")

# # Extract the parts before and after the specified indices
# filtered_text = full_text[:start_index] + full_text[end_index + len("[/INST]"):]

# print(filtered_text)

In [8]:
!pip install chainlit
!pip install ctransformers
!pip install torch
!pip install sentence_transformers
!pip install chromadb
!pip install langchain
!pip install pypdf
!pip install PyPDF2

In [9]:
import os
import torch
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader

# Check if CUDA is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "BAAI/bge-large-en"
model_kwargs = {'device': device}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

loader = PyPDFLoader("/content/IntroductionAIPH.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

# Move embeddings to the device
# embeddings.model.to(device)

# Create Chroma instance without device argument
vector_store = Chroma.from_documents(
    texts,
    embeddings,
    collection_metadata={"hnsw:space": "cosine"},
    persist_directory="stores/PhySIO_cosineProM"
)

print("Vector Store Created.......")


Vector Store Created.......


In [10]:
# pip install ctransformers[gptq]

In [11]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import CTransformers
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceBgeEmbeddings
from io import BytesIO
from langchain.document_loaders import PyPDFLoader
# import gradio as gr
import time

local_llm = "/content/zephyr-7B-beta-GPTQ"

# local_llm = GPT2LMHeadModel.from_pretrained(""TheBloke/zephyr-7B-beta-GGUF", model_file="zephyr-7b-beta.Q5_K_M.gguf", model_type="mistral", gpu_layers=5")
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")




config = {
'max_new_tokens': 1024,
'repetition_penalty': 1.1,
'temperature': 0.1,
'top_k': 50,
'top_p': 0.9,
'stream': True,
'threads': int(os.cpu_count()/2)
}

llm = CTransformers(
    model=local_llm,
    lib="avx2",  # for CPU use
    device="device",
    **config
)

print("LLM Initialized...")


prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

model_name = "BAAI/bge-large-en"
model_kwargs = {'device': "cpu"}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


prompt = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])



load_vector_store = Chroma(persist_directory="/content/stores/PhySIO_cosineProM", embedding_function=embeddings)
retriever = load_vector_store.as_retriever(search_kwargs={"k":5},search_type="similarity")
query = "what are benefits of physical therapy?"
semantic_search = retriever.get_relevant_documents(query)
print(semantic_search)

LLM Initialized...
[Document(page_content="routine. These measures can help improve blood circulation and alleviate symptoms. It's \nimportant to avoid prolonged peri ods of sitting or standing, as this can contribute to \ndiscomfort. For personalized advice and to explore additional treatment options, consult \nwith a healthcare professional or a vascular specialist to address your specific cond ition.  \n \n \n• Knee Pain Relief During Stair Climbing  To alleviate knee pain when climbing", metadata={'page': 6, 'source': '/content/IntroductionAIPH.pdf'}), Document(page_content="routine. These measures can help improve blood circulation and alleviate symptoms. It's \nimportant to avoid prolonged peri ods of sitting or standing, as this can contribute to \ndiscomfort. For personalized advice and to explore additional treatment options, consult \nwith a healthcare professional or a vascular specialist to address your specific cond ition.  \n \n \n• Knee Pain Relief During Stair Climbing 

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
from transformers import pipeline

clf = pipeline('sentiment-analysis',
               model='/content/drive/MyDrive/models/MSR_3',
               return_all_scores=True)

prediction = clf('my back is hurting very bad so can you tell me what is the treatment')

print(prediction)

[[{'label': 'LABEL_0', 'score': 0.0011447904398664832}, {'label': 'LABEL_1', 'score': 0.9969103932380676}, {'label': 'LABEL_2', 'score': 0.0019447808153927326}]]


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [14]:
# Define a dictionary of label names
label_names = {0: 'Report', 1: 'Phy', 2: 'Greets'}

# Extract predictions from the output
output = clf('I have a stiff neck and shoulders. Can you recommend stretches to help relieve the stiffness?')
predictions = output[0]
# Loop through each prediction and print the label name and score
for prediction in predictions:
    label_idx = int(prediction['label'].split('_')[-1])  # Extract label index from the label name
    label_name = label_names[label_idx]  # Look up label name from dictionary
    score = prediction['score']
    print(f"Predicted label: {label_name}, Score: {score}")


Predicted label: Report, Score: 0.000624738575424999
Predicted label: Phy, Score: 0.9985232949256897
Predicted label: Greets, Score: 0.0008520885603502393


In [15]:
prediction = clf('I want take my medical reports')
print(prediction)

[[{'label': 'LABEL_0', 'score': 0.9272016286849976}, {'label': 'LABEL_1', 'score': 0.012927361764013767}, {'label': 'LABEL_2', 'score': 0.05987099930644035}]]


In [16]:
print("######################################################################")
start_time = time.time()
chain_type_kwargs = {"prompt": prompt}

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents = True,
    chain_type_kwargs= chain_type_kwargs,
    verbose=True
)

response = qa(query)

print(response)


# chain_type_kwargs = {
#     "prompt": prompt,
#     "previous_question": "Previous question goes here",
#     "previous_answer": "Previous answer goes here"
# }

# qa = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=retriever,
#     return_source_documents=True,
#     chain_type_kwargs=chain_type_kwargs,
#     verbose=True
# )

# response = qa(query)

######################################################################


> Entering new RetrievalQA chain...

> Finished chain.
{'query': 'what are benefits of physical therapy?', 'result': 'Physical therapy can help improve strength, flexibility, and range of motion in affected areas, reducing pain and promoting overall mobility and function. It may also address underlying conditions or injuries that contribute to discomfort or dysfunction, working in conjunction with other medical treatments as part of a comprehensive care plan. Consult with a healthcare professional or physical therapist for personalized guidance and advice.', 'source_documents': [Document(page_content="routine. These measures can help improve blood circulation and alleviate symptoms. It's \nimportant to avoid prolonged peri ods of sitting or standing, as this can contribute to \ndiscomfort. For personalized advice and to explore additional treatment options, consult \nwith a healthcare professional or a vascular sp

In [17]:
import time
sample = ["Hello!How are you?","Who are you ?","My hand is having pain .What can be the issue?", "My back hurting very back specially lower back.What can be the issue ?","Can you suggest me any exercise for it?","I am feeling stress in my legs.What can be the issue? and its cure","ok then what will be exercise for shoulder comfort","My heel hurts when I walk. What can I do about it?","My calf is tight. What's the best way to stretch it?","Experiencing pain due to flat feet. What shoes are best for me?","thanks for your help"]
start =time.time()
for i in range(len(sample)):

    response = qa(sample[i])
    print("Human: ")
    print(response['query'])
    print("Physio_Bot: ")
    print(response['result'])


end = time.time()

print("Seconds Taken :",start-end)



> Entering new RetrievalQA chain...

> Finished chain.
Human: 
Hello!How are you?
Physio_Bot: 
I'm doing well, thank you. How can I assist you today? Let's get started!


> Entering new RetrievalQA chain...

> Finished chain.
Human: 
Who are you ?
Physio_Bot: 
I am an AI Physiotherapist here to assist with pain points and physical disturbances. You can ask me any relevant questions.


> Entering new RetrievalQA chain...

> Finished chain.
Human: 
My hand is having pain .What can be the issue?
Physio_Bot: 

The cause of your hand pain could vary, from a repetitive strain injury to arthritis or an underlying medical condition. As it's important to diagnose and treat the issue properly, we recommend consulting with a healthcare professional for evaluation and guidance. In the meantime, resting initially but avoiding prolonged inactivity and engaging in gentle hand exercises and stretches under the guidance of a physiotherapist may help alleviate symptoms. Remember to avoid self-diagnosi

In [18]:
# pip show gradio

In [19]:
# pip show pydantic

In [20]:
import gradio as gr

def echo(text_input):
  x = qa(text_input)


  return x['result']

iface = gr.Interface(fn=echo, inputs="text", outputs="text")
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e0306cca22e3bcb771.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
